In [230]:
import os
import pickle
import numpy as np
import keras.backend as K
import tensorflow as tf

from glob import glob
from copy import deepcopy
from collections import OrderedDict
from keras.models import load_model, clone_model
from keras.layers import Input

In [2]:
g_config = tf.ConfigProto()
g_config.gpu_options.per_process_gpu_memory_fraction = 0.4
sess = tf.Session()
K.set_session(sess)

In [3]:
BASE_DIR = '../data/'

In [4]:
init_model = load_model('../log/grad/init-model.hdf5')

In [5]:
last_model = load_model('../log/grad/last-model.hdf5')

In [6]:
X_train = np.load(os.path.join(BASE_DIR, "X_train.npy"))
y_train = np.load(os.path.join(BASE_DIR, "y_train.npy"))
X_test = np.load(os.path.join(BASE_DIR, "X_test.npy"))
y_test = np.load(os.path.join(BASE_DIR, "y_test.npy"))

mean = X_train.mean(axis=(0, 1, 2))
std = X_train.std(axis=(0, 1, 2))

X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

In [7]:
init_model_variable = {}

In [8]:
for layer in init_model.layers:
    if len(layer.weights) <= 0:
        continue
    print(layer.name)
    init_model_variable.update({layer.name: layer.get_weights()})

conv_first
stage1_0_bn0
stage1_0_conv0
stage1_0_bn1
stage1_0_conv1
stage1_0_convdiff
stage1_1_bn0
stage1_1_conv0
stage1_1_bn1
stage1_1_conv1
stage1_2_bn0
stage1_2_conv0
stage1_2_bn1
stage1_2_conv1
stage1_3_bn0
stage1_3_conv0
stage1_3_bn1
stage1_3_conv1
stage2_0_bn0
stage2_0_conv0
stage2_0_bn1
stage2_0_conv1
stage2_0_convdiff
stage2_1_bn0
stage2_1_conv0
stage2_1_bn1
stage2_1_conv1
stage2_2_bn0
stage2_2_conv0
stage2_2_bn1
stage2_2_conv1
stage2_3_bn0
stage2_3_conv0
stage2_3_bn1
stage2_3_conv1
stage3_0_bn0
stage3_0_conv0
stage3_0_bn1
stage3_0_conv1
stage3_0_convdiff
stage3_1_bn0
stage3_1_conv0
stage3_1_bn1
stage3_1_conv1
stage3_2_bn0
stage3_2_conv0
stage3_2_bn1
stage3_2_conv1
stage3_3_bn0
stage3_3_conv0
stage3_3_bn1
stage3_3_conv1
dense_1


In [9]:
last_model_variable = {}

In [10]:
for layer in last_model.layers:
    if len(layer.weights) <= 0:
        continue
    print(layer.name)
    last_model_variable.update({layer.name: layer.get_weights()})

conv_first
stage1_0_bn0
stage1_0_conv0
stage1_0_bn1
stage1_0_conv1
stage1_0_convdiff
stage1_1_bn0
stage1_1_conv0
stage1_1_bn1
stage1_1_conv1
stage1_2_bn0
stage1_2_conv0
stage1_2_bn1
stage1_2_conv1
stage1_3_bn0
stage1_3_conv0
stage1_3_bn1
stage1_3_conv1
stage2_0_bn0
stage2_0_conv0
stage2_0_bn1
stage2_0_conv1
stage2_0_convdiff
stage2_1_bn0
stage2_1_conv0
stage2_1_bn1
stage2_1_conv1
stage2_2_bn0
stage2_2_conv0
stage2_2_bn1
stage2_2_conv1
stage2_3_bn0
stage2_3_conv0
stage2_3_bn1
stage2_3_conv1
stage3_0_bn0
stage3_0_conv0
stage3_0_bn1
stage3_0_conv1
stage3_0_convdiff
stage3_1_bn0
stage3_1_conv0
stage3_1_bn1
stage3_1_conv1
stage3_2_bn0
stage3_2_conv0
stage3_2_bn1
stage3_2_conv1
stage3_3_bn0
stage3_3_conv0
stage3_3_bn1
stage3_3_conv1
dense_1


In [11]:
diff_model_variable = {}

In [85]:
for key in last_model_variable.keys():
    src_variable = init_model_variable[key]
    dst_variable = last_model_variable[key]
    diff = []
    for s, d in zip(src_variable, dst_variable):
        diff.append(d - s)
    diff_model_variable.update({key: diff})

In [110]:
diff_50_div_variable = {}

In [111]:
for key in diff_model_variable.keys():
    diff = []
    for v in diff_model_variable[key]:
        diff.append(v / 50)
    diff_50_div_variable.update({key: diff})

In [112]:
for key in diff_50_div_variable:
    i_weights = init_model_variable[key]
    l_weights = last_model_variable[key]
    d_weights = diff_50_div_variable[key]
    for i, l, d in zip(i_weights, l_weights, d_weights):
        d = d.copy() * 50
        new_i = i + d
        if not np.all(np.isclose(l, new_i, atol=1e-6)):
            print('NG')

stage2_2_conv0
conv_first
stage2_1_bn0
stage2_0_bn0
stage3_3_bn0
stage3_3_bn1
stage2_3_bn0
stage1_2_bn1
stage2_3_conv0
stage1_3_conv0
stage2_0_bn1
stage3_0_bn1
stage1_0_conv1
stage1_2_conv0
stage2_1_conv1
stage2_2_bn1
stage1_1_bn1
stage2_0_convdiff
stage2_1_bn1
stage3_1_bn0
stage1_0_convdiff
stage3_1_conv0
stage1_2_bn0
stage3_0_convdiff
stage2_2_bn0
stage1_0_bn1
stage1_0_bn0
stage1_0_conv0
stage3_0_conv1
stage3_2_conv0
stage2_1_conv0
stage1_1_bn0
stage3_1_conv1
stage1_2_conv1
stage2_0_conv0
stage3_1_bn1
stage2_3_conv1
stage3_0_bn0
stage3_2_conv1
stage3_3_conv1
stage2_0_conv1
stage3_0_conv0
stage3_2_bn1
stage3_2_bn0
stage1_3_conv1
stage1_3_bn0
stage1_3_bn1
stage1_1_conv1
stage2_3_bn1
stage3_3_conv0
dense_1
stage1_1_conv0
stage2_2_conv1


In [229]:
def set_model_variable(src_model, dst_model, diff_variable_dict, i):
    # TODO 折角作成したinit_variable_dictを利用して高速化を図る．
    for key in diff_variable_dict.keys():
        src_layer = src_model.get_layer(key)
        dst_layer = dst_model.get_layer(key)
        src_weights = src_layer.get_weights()
        add_weights = diff_variable_dict[key]
        new_weights = []
        for s, a in zip(src_weights, add_weights):
            dst = s + a * i
            new_weights.append(dst)
        dst_layer.set_weights(new_weights)

In [226]:
loss = []
tmp_model = clone_model(init_model)
tmp_model.set_weights(init_model.get_weights())
op_loss = [o for o in tmp_model.losses]
fn = K.function(tmp_model.inputs, op_loss, updates=False)

In [227]:
for i in range(1, 51):
    print('{0}番目の処理'.format(i))
    set_model_variable(init_model, tmp_model, diff_50_div_variable, i)
    loss.append(np.sum(fn([X_train])))

1番目の処理
2番目の処理
3番目の処理
4番目の処理
5番目の処理
6番目の処理
7番目の処理
8番目の処理
9番目の処理
10番目の処理
11番目の処理
12番目の処理
13番目の処理
14番目の処理
15番目の処理
16番目の処理
17番目の処理
18番目の処理
19番目の処理
20番目の処理
21番目の処理
22番目の処理
23番目の処理
24番目の処理
25番目の処理
26番目の処理
27番目の処理
28番目の処理
29番目の処理
30番目の処理
31番目の処理
32番目の処理
33番目の処理
34番目の処理
35番目の処理
36番目の処理
37番目の処理
38番目の処理
39番目の処理
40番目の処理
41番目の処理
42番目の処理
43番目の処理
44番目の処理
45番目の処理
46番目の処理
47番目の処理
48番目の処理
49番目の処理
50番目の処理


In [231]:
with open('../log/grad/loss.pkl', 'wb') as f:
    pickle.dump(loss, f)